In [1]:
import pandas as pd
import io
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
import pickle
from scipy import stats
print ('Ran')

Ran


In [2]:
app_train = pd.read_csv('D:/Project_Data/application_train.csv')
prev_app = pd.read_csv('D:/Project_Data/previous_application.csv')

In [3]:
cat_features = [f for f in prev_app.columns if prev_app[f].dtype == 'object']

In [4]:
categorical_var = [x for x in prev_app.columns if x in cat_features + ['SK_ID_CURR']]

In [5]:
continuous_var = [x for x in prev_app.columns if x not in categorical_var]

In [6]:
agg_previous_application_1 = prev_app.groupby('SK_ID_CURR').agg({k: lambda x: stats.mode(x)[0] for k in categorical_var})

C:\Users\dbda\Anaconda3\lib\site-packages\scipy\stats\stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [7]:
agg_previous_application_1.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,WEEKDAY_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,NAME_SELLER_INDUSTRY,NAME_YIELD_GROUP,PRODUCT_COMBINATION
SK_ID_CURR,,,,,,,,,,,,,,,,,
100001,100001,Consumer loans,FRIDAY,Y,XAP,Approved,Cash through the bank,XAP,Family,Refreshed,Mobile,POS,XNA,Country-wide,Connectivity,high,POS mobile with interest
100002,100002,Consumer loans,SATURDAY,Y,XAP,Approved,XNA,XAP,0,New,Vehicles,POS,XNA,Stone,Auto technology,low_normal,POS other with interest
100003,100003,Consumer loans,FRIDAY,Y,XAP,Approved,Cash through the bank,XAP,Family,Refreshed,Consumer Electronics,POS,XNA,Country-wide,Consumer electronics,middle,Cash X-Sell: low
100004,100004,Consumer loans,FRIDAY,Y,XAP,Approved,Cash through the bank,XAP,Unaccompanied,New,Mobile,POS,XNA,Regional / Local,Connectivity,middle,POS mobile without interest
100005,100005,Cash loans,FRIDAY,Y,XAP,Approved,Cash through the bank,XAP,0,New,Mobile,POS,XNA,Country-wide,Connectivity,XNA,Cash


In [8]:
agg_previous_application_2 = prev_app.groupby('SK_ID_CURR').agg({k:lambda x: x.mean() for k in continuous_var})

In [9]:
agg_previous_application_2.shape

(338857, 20)

In [10]:
agg_previous_application_1.head(20)

,SK_ID_CURR,NAME_CONTRACT_TYPE,WEEKDAY_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,NAME_SELLER_INDUSTRY,NAME_YIELD_GROUP,PRODUCT_COMBINATION
SK_ID_CURR,,,,,,,,,,,,,,,,,
100001,100001,Consumer loans,FRIDAY,Y,XAP,Approved,Cash through the bank,XAP,Family,Refreshed,Mobile,POS,XNA,Country-wide,Connectivity,high,POS mobile with interest
100002,100002,Consumer loans,SATURDAY,Y,XAP,Approved,XNA,XAP,0,New,Vehicles,POS,XNA,Stone,Auto technology,low_normal,POS other with interest
100003,100003,Consumer loans,FRIDAY,Y,XAP,Approved,Cash through the bank,XAP,Family,Refreshed,Consumer Electronics,POS,XNA,Country-wide,Consumer electronics,middle,Cash X-Sell: low
100004,100004,Consumer loans,FRIDAY,Y,XAP,Approved,Cash through the bank,XAP,Unaccompanied,New,Mobile,POS,XNA,Regional / Local,Connectivity,middle,POS mobile without interest
100005,100005,Cash loans,FRIDAY,Y,XAP,Approved,Cash through the bank,XAP,0,New,Mobile,POS,XNA,Country-wide,Connectivity,XNA,Cash
100006,100006,Cash loans,THURSDAY,Y,XNA,Approved,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,XNA,Credit and cash offices,XNA,XNA,Cash
100007,100007,Cash loans,SUNDAY,Y,XNA,Approved,Cash through the bank,XAP,Family,Repeater,XNA,Cash,x-sell,Country-wide,Consumer electronics,high,Cash X-Sell: middle
100008,100008,Consumer loans,MONDAY,Y,XAP,Approved,Cash through the bank,XAP,Unaccompanied,Repeater,XNA,POS,XNA,Country-wide,Consumer electronics,low_normal,POS household with interest
100009,100009,Consumer loans,SATURDAY,Y,XAP,Approved,Cash through the bank,XAP,Family,Repeater,Audio/Video,POS,XNA,Regional / Local,Consumer electronics,middle,POS household with interest


In [11]:
agg_previous_application_2.head(20)

,SK_ID_PREV,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,
100001,1.369693e+06,3951.000000,24835.500000,23787.000000,2520.000000,24835.500000,13.000000,1.0,0.104326,NaN,NaN,-1740.000000,23.000000,8.000000,365243.000000,-1709.000000,-1499.000000,-1619.000000,-1612.000000,0.000000
100002,1.038818e+06,9251.775000,179055.000000,179055.000000,0.000000,179055.000000,9.000000,1.0,0.000000,NaN,NaN,-606.000000,500.000000,24.000000,365243.000000,-565.000000,125.000000,-25.000000,-17.000000,0.000000
100003,2.281150e+06,56553.990000,435436.500000,484191.000000,3442.500000,435436.500000,14.666667,1.0,0.050030,NaN,NaN,-1305.000000,533.000000,10.000000,365243.000000,-1274.333333,-1004.333333,-1054.333333,-1047.333333,0.666667
100004,1.564014e+06,5357.250000,24282.000000,20106.000000,4860.000000,24282.000000,5.000000,1.0,0.212008,NaN,NaN,-815.000000,30.000000,4.000000,365243.000000,-784.000000,-694.000000,-724.000000,-714.000000,0.000000
100005,2.176837e+06,4813.200000,22308.750000,20076.750000,4464.000000,44617.500000,10.500000,1.0,0.108964,NaN,NaN,-536.000000,18.000000,12.000000,365243.000000,-706.000000,-376.000000,-466.000000,-460.000000,0.000000
100006,1.932462e+06,23651.175000,272203.260000,291695.500000,34840.170000,408304.890000,14.666667,1.0,0.163412,NaN,NaN,-272.444444,894.222222,23.000000,365243.000000,91066.500000,91584.000000,182477.500000,182481.750000,0.000000
100007,2.157812e+06,12278.805000,150530.250000,166638.750000,3390.750000,150530.250000,12.333333,1.0,0.159516,NaN,NaN,-1222.833333,409.166667,20.666667,365243.000000,-1263.200000,-837.200000,72136.200000,72143.800000,0.600000
100008,1.936735e+06,15839.696250,155701.800000,162767.700000,5548.500000,194627.250000,12.000000,1.0,0.073051,NaN,NaN,-1192.000000,73.000000,14.000000,365243.000000,-1434.500000,-1044.500000,-1209.500000,-872.750000,0.250000
100009,1.881798e+06,10051.412143,76741.714286,70137.642857,9203.142857,76741.714286,13.714286,1.0,0.126602,NaN,NaN,-719.285714,170.000000,8.000000,365243.000000,-688.285714,-478.285714,51666.857143,51672.857143,0.000000


In [12]:
prev_app=pd.merge(agg_previous_application_1, agg_previous_application_2, left_index=True, right_index=True)

In [13]:
prev_app.head(20)

,SK_ID_CURR,NAME_CONTRACT_TYPE,WEEKDAY_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,...,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,100001,Consumer loans,FRIDAY,Y,XAP,Approved,Cash through the bank,XAP,Family,Refreshed,...,NaN,-1740.000000,23.000000,8.000000,365243.000000,-1709.000000,-1499.000000,-1619.000000,-1612.000000,0.000000
100002,100002,Consumer loans,SATURDAY,Y,XAP,Approved,XNA,XAP,0,New,...,NaN,-606.000000,500.000000,24.000000,365243.000000,-565.000000,125.000000,-25.000000,-17.000000,0.000000
100003,100003,Consumer loans,FRIDAY,Y,XAP,Approved,Cash through the bank,XAP,Family,Refreshed,...,NaN,-1305.000000,533.000000,10.000000,365243.000000,-1274.333333,-1004.333333,-1054.333333,-1047.333333,0.666667
100004,100004,Consumer loans,FRIDAY,Y,XAP,Approved,Cash through the bank,XAP,Unaccompanied,New,...,NaN,-815.000000,30.000000,4.000000,365243.000000,-784.000000,-694.000000,-724.000000,-714.000000,0.000000
100005,100005,Cash loans,FRIDAY,Y,XAP,Approved,Cash through the bank,XAP,0,New,...,NaN,-536.000000,18.000000,12.000000,365243.000000,-706.000000,-376.000000,-466.000000,-460.000000,0.000000
100006,100006,Cash loans,THURSDAY,Y,XNA,Approved,XNA,XAP,Unaccompanied,Repeater,...,NaN,-272.444444,894.222222,23.000000,365243.000000,91066.500000,91584.000000,182477.500000,182481.750000,0.000000
100007,100007,Cash loans,SUNDAY,Y,XNA,Approved,Cash through the bank,XAP,Family,Repeater,...,NaN,-1222.833333,409.166667,20.666667,365243.000000,-1263.200000,-837.200000,72136.200000,72143.800000,0.600000
100008,100008,Consumer loans,MONDAY,Y,XAP,Approved,Cash through the bank,XAP,Unaccompanied,Repeater,...,NaN,-1192.000000,73.000000,14.000000,365243.000000,-1434.500000,-1044.500000,-1209.500000,-872.750000,0.250000
100009,100009,Consumer loans,SATURDAY,Y,XAP,Approved,Cash through the bank,XAP,Family,Repeater,...,NaN,-719.285714,170.000000,8.000000,365243.000000,-688.285714,-478.285714,51666.857143,51672.857143,0.000000


In [14]:
app_train.shape

(307511, 122)

In [15]:
prev_app.shape

(338857, 37)

In [16]:
prev_app.drop(['SK_ID_PREV'],axis=1,inplace=True)

In [17]:
data_train = app_train.merge(prev_app,how='left', left_on='SK_ID_CURR',right_index=True)

In [18]:
data_train.shape

(307511, 158)

In [19]:
data_train.to_csv('merge_train_prev.csv', index = False)